In [8]:
import numpy as np
%matplotlib qt5
import matplotlib.pyplot as pl
import os
import pickle


In [2]:
os.getcwd()

'/Users/utkarshagrawal/Documents/Postdoc/U_1_exp/Weak measurements'

In [9]:

def load_data(filename='nothing_fixed'):

    with open(filename,'rb') as f:
        data = pickle.load(f)

    return data


def plot_success_ent(data,L_list,charge_fac = 0,N_samples=-1,marker='o'):
    ent = {}
    err = {}
    for L in L_list[:]:
        ent[L] = []
        err[L] = []
        p_list = list(data[L].keys())
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent.append(np.sum(np.array(suc_list)>0.5)/len(suc_list))
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])
            ent[L].append(np.average(ent_list))
            err[L].append(np.std(ent_list)/(len(ent_list)-1)**0.5)

        pl.errorbar(np.array(p_list)/(np.pi/2),ent[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    # pl.ylabel(r'$\langle p \rangle_{suc}$',fontsize=16)
    pl.ylabel(r'$ p_{suc}$',fontsize=16)

    pl.legend(fontsize=16)
    pl.tight_layout()




In [10]:
def plot_success_binder(data,L_list,N_samples=-1,marker='o'):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        p_list = np.array(list(data[L].keys()))
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
            # binder[L].append(mu_2)
            err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2))*0)
        
        pl.errorbar(p_list/(np.pi/2),binder[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Binder${}_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

def plot_acc_binder(data,L_list,N_samples=-1,marker='-o'):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        p_list = np.array(list(data[L].keys()))
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([x for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
            err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2)))

        pl.errorbar(p_list/(np.pi/2),binder[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Binder${}_{acc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

In [5]:
data=load_data('sep_data/seed=1_all_qubits_special_scrambled')
pl.plot(data[10][0.571][4])

In [17]:
%matplotlib qt
pl.figure(1)

data=load_data('data/sep_data/seed=1_all_qubits_special_scrambled')
L_list = [6,8,10,12,14,16][1::2]
# plot_success_ent(data,L_list,charge_fac=0,marker='s')
plot_success_binder(data,L_list,marker='s',N_samples=300)
# pl.ylim(0,np.log(2))
pl.title('Not Noisy, N_samples = 300',fontsize=16)
pl.tight_layout()

pl.figure(2)
# pl.ylim(0,np.log(2))
data=load_data('data/biased_sep_data/seed=1_all_qubits_special_scrambled_noisy')
L_list = [6,8,10,12,14,16][1::2]
# plot_success_ent(data,L_list,charge_fac=0,marker='s')
plot_success_binder(data,L_list,marker='s',N_samples=300)
pl.title('Noisy, N_samples = 300',fontsize=16)
pl.tight_layout()

L= 8 p= 0.2 data_size: 300 300
L= 8 p= 0.253 data_size: 300 300
L= 8 p= 0.306 data_size: 300 300
L= 8 p= 0.359 data_size: 300 300
L= 8 p= 0.412 data_size: 300 300
L= 8 p= 0.465 data_size: 300 300
L= 8 p= 0.518 data_size: 300 300
L= 8 p= 0.571 data_size: 300 300
L= 8 p= 0.624 data_size: 300 300
L= 8 p= 0.677 data_size: 300 300
L= 8 p= 0.73 data_size: 300 300
L= 8 p= 0.783 data_size: 300 300
L= 8 p= 0.836 data_size: 300 300
L= 8 p= 0.889 data_size: 300 300
L= 8 p= 0.942 data_size: 300 300
L= 12 p= 0.2 data_size: 300 300
L= 12 p= 0.253 data_size: 300 300
L= 12 p= 0.306 data_size: 300 300
L= 12 p= 0.359 data_size: 300 300
L= 12 p= 0.412 data_size: 300 300
L= 12 p= 0.465 data_size: 300 300
L= 12 p= 0.518 data_size: 300 300
L= 12 p= 0.571 data_size: 300 300
L= 12 p= 0.624 data_size: 300 300
L= 12 p= 0.677 data_size: 300 300
L= 12 p= 0.73 data_size: 300 300
L= 12 p= 0.783 data_size: 300 300
L= 12 p= 0.836 data_size: 300 300
L= 12 p= 0.889 data_size: 300 300
L= 12 p= 0.942 data_size: 300 300
L

In [15]:
%matplotlib qt
pl.figure(1)

data=load_data('data/sep_data/seed=1_all_qubits_special_scrambled')
L_list = [6,8,10,12,14,16][:]
plot_success_ent(data,L_list,charge_fac=0,marker='s',N_samples=300)
# pl.ylim(0,np.log(2))
pl.title('Not noisy, N_samples = 200',fontsize=16)
pl.tight_layout()

pl.figure(2)
# pl.ylim(0,np.log(2))
data=load_data('data/sep_data/seed=1_all_qubits_special_scrambled_noisy')
L_list = [6,8,10,12,14,16][:]
plot_success_ent(data,L_list,charge_fac=0,marker='s',N_samples=300)
pl.title('Noisy, N_samples = 200',fontsize=16)
pl.tight_layout()

L= 6 p= 0.2 data_size: 300 300
L= 6 p= 0.253 data_size: 300 300
L= 6 p= 0.306 data_size: 300 300
L= 6 p= 0.359 data_size: 300 300
L= 6 p= 0.412 data_size: 300 300
L= 6 p= 0.465 data_size: 300 300
L= 6 p= 0.518 data_size: 300 300
L= 6 p= 0.571 data_size: 300 300
L= 6 p= 0.624 data_size: 300 300
L= 6 p= 0.677 data_size: 300 300
L= 6 p= 0.73 data_size: 300 300
L= 6 p= 0.783 data_size: 300 300
L= 6 p= 0.836 data_size: 300 300
L= 6 p= 0.889 data_size: 300 300
L= 6 p= 0.942 data_size: 300 300
L= 8 p= 0.2 data_size: 300 300
L= 8 p= 0.253 data_size: 300 300
L= 8 p= 0.306 data_size: 300 300
L= 8 p= 0.359 data_size: 300 300
L= 8 p= 0.412 data_size: 300 300
L= 8 p= 0.465 data_size: 300 300
L= 8 p= 0.518 data_size: 300 300
L= 8 p= 0.571 data_size: 300 300
L= 8 p= 0.624 data_size: 300 300
L= 8 p= 0.677 data_size: 300 300
L= 8 p= 0.73 data_size: 300 300
L= 8 p= 0.783 data_size: 300 300
L= 8 p= 0.836 data_size: 300 300
L= 8 p= 0.889 data_size: 300 300
L= 8 p= 0.942 data_size: 300 300
L= 10 p= 0.2 dat

In [21]:
# Function to ensure that p_suc for initial states with charge Q has non-zero probabilities
def sanity_check(data,L_list,p_list):
    for L in L_list[:]:
        for p in p_list:
            p_Q = list(data[L][p][L//2])
            p_Q2 = list(data[L][p][L//2-1])
            if np.any(p_Q==0) or np.any(p_Q2==0):
                print("There are some zero probabilities")
            else:
                print("L =",L,"p =",p,"Sanity check complete; no zero probabilities")

In [10]:
sanity_check(data,L_list,p_list)

L = 8 p = 0.05 Sanity check complete; no zero probabilities
L = 8 p = 0.1 Sanity check complete; no zero probabilities
L = 8 p = 0.13 Sanity check complete; no zero probabilities
L = 8 p = 0.16 Sanity check complete; no zero probabilities
L = 8 p = 0.2 Sanity check complete; no zero probabilities
L = 8 p = 0.25 Sanity check complete; no zero probabilities
L = 8 p = 0.3 Sanity check complete; no zero probabilities
L = 10 p = 0.05 Sanity check complete; no zero probabilities
L = 10 p = 0.1 Sanity check complete; no zero probabilities
L = 10 p = 0.13 Sanity check complete; no zero probabilities
L = 10 p = 0.16 Sanity check complete; no zero probabilities
L = 10 p = 0.2 Sanity check complete; no zero probabilities
L = 10 p = 0.25 Sanity check complete; no zero probabilities
L = 10 p = 0.3 Sanity check complete; no zero probabilities
L = 12 p = 0.05 Sanity check complete; no zero probabilities
L = 12 p = 0.1 Sanity check complete; no zero probabilities
L = 12 p = 0.13 Sanity check complete;